In [1]:
# 라이브러리 호출
import os
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bts
import json
import re
import time
from tqdm.notebook import tqdm

In [2]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\2-13\\Desktop\\DMF\\final\\naver'

In [3]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['.ipynb_checkpoints',
 'crawling.py',
 'data',
 'Naver_News_List.pkl',
 'Naver_News_List_2.pkl',
 'naver_news_samsung.csv',
 'SK하이닉스크롤링.ipynb',
 'SK하이닉스크롤링step2.ipynb',
 '삼전 크롤링 step2.ipynb',
 '삼전크롤링.ipynb']

In [4]:
# pkl 파일을 읽고 newsList 생성
newsList = pd.read_pickle(filepath_or_buffer = 'Naver_News_List_2.pkl')

In [5]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date
759,연합뉴스,"HBM 날개 단 SK하이닉스, 3분기 적자 줄였다…D램은 흑자전환(종합)",https://n.news.naver.com/mnews/article/001/001...,2023.10.26.
760,노컷뉴스,"SK하이닉스, 3Q 영업손실 1.8조…적자 '1조' 줄였다",https://n.news.naver.com/mnews/article/079/000...,2023.10.26.
761,파이낸셜뉴스,[상보] SK하이닉스 D램 흑자전환... 적자폭 1兆 감소,https://n.news.naver.com/mnews/article/014/000...,2023.10.26.
762,서울신문,[속보] SK하이닉스 3분기 영업손실 1조 8000억원…D램은 흑자 전환,https://n.news.naver.com/mnews/article/081/000...,2023.10.26.
763,한국경제,SK하이닉스 D램 2분기 만에 흑자전환…3분기 영업손실 1.8조원,https://n.news.naver.com/mnews/article/015/000...,2023.10.26.


In [6]:
# newsList의 정보 확인
newsList.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   press   764 non-null    object
 1   title   764 non-null    object
 2   nlink   764 non-null    object
 3   date    764 non-null    object
dtypes: object(4)
memory usage: 24.0+ KB


In [7]:
# 네이버뉴스 링크로 HTTP 요청을 실행하고 뉴스 본문을 반환하는 함수 생성
def NaverNewsBody(nlink):
    
    # nlink에 따라 HTTP 요청 실행 및 응답 바디 문자열 처리 코드 분기
    if 'https://n.news' in nlink:
        
        # 네이버뉴스 링크로 HTTP 요청 실행
        res = requests.get(url = nlink)
        
        # HTTP 응답 바디 문자열을 bs4.BeautifulSoup 객체로 변환
        soup = bts(markup = res.text, features = 'html.parser')
        
        # soup에서 뉴스 본문을 포함하는 HTML 요소 선택
        item = soup.select_one(selector = '#dic_area, div._article_content')
        
        # item의 텍스트 반환
        # [참고] strip()은 문자열 양 옆에 있는 공백을 모두 제거함
        return item.text.strip()
    
    elif 'https://m' in nlink:
        
        # 'https://m'으로 시작하는 nlink의 일부 문자열 변경한 url 생성
        url = re.sub(pattern = 'https://m.(sports|entertain).naver.com(.+)?(?=/article)', 
                     repl = r'https://api-gw.\1.naver.com/news', 
                     string = newsList['nlink'].iloc[i])
        
        # url로 HTTP 요청 실행
        res = requests.get(url = url)
        
        # JSON 형태의 문자열을 딕셔너리로 변환
        dic = json.loads(s = res.text)
        
        # 뉴스 본문 반환
        return dic['result']['articleInfo']['article']['content']

In [8]:
# 인덱스 설정: 첫 번째 원소
# [참고] 반복문으로 실행할 코드에 인덱스를 추가하면 편리함
i = 0

In [9]:
# 해당 인덱스의 nlink 확인
print(newsList['nlink'].iloc[i])

https://n.news.naver.com/mnews/article/421/0007116233


In [10]:
# 함수 테스트
NaverNewsBody(nlink = newsList['nlink'].iloc[i])

'기존 속도 기준에 \'성능밀도\' 추가…엔비디아 저사양 AI칩 목표제3국 우회한 중국으로 반도체 유입 막기 위해 수출통제 대상국 확대\n\n\n\n지나 러몬도 미국 상무장관. 2023.8.30 ⓒ AFP=뉴스1 ⓒ News1 우동명 기자(워싱턴·서울=뉴스1) 강민경 기자 김현 특파원 = 미국 정부가 인공지능(AI) 반도체의 대중국 수출 통제를 강화한다. 이전 대중국 수출통제 조치 때보다 사양이 낮은 인공지능(AI) 칩에 대해서도 수출을 추가 금지한다. 또한 중국의 제재 우회를 막기 위해 마카오에 본사가 있거나, 미국의 무기금수 조치 대상인 회사로 반도체 장비 등을 수출하는 것도 통제된다.로이터통신과 뉴욕타임스(NYT) 등 외신들에 따르면 상무부는 17일(현지시간) 이같은 내용이 포함된 대중국 반도체 수출통제 조치를 추가로 발표했다.앞서 상무부는 지난해 10월 고성능 AI 학습용칩, 슈퍼컴퓨터용 특정 반도체 등을 포함한 첨단 컴퓨팅 반도체칩, 슈퍼컴퓨터용 반도체칩 거래 등에 대해 수출제한 조치를 부과한 바 있다.  미국 정부는 중국이 이같은 첨단 반도체에 접근할 경우 중국군의 극초음속 미사일 개발, 첨단 정찰 시스템 구축, 미국의 극비암호 해독 등에 사용돼 미국의 국가안보를 위협할 수 있기 때문에 이같은 수출통제를 부과한다고 설명했다.    그러나 엔비디아 등 일부 반도체 기업들이 기존 수출통제 조치에 포함되지 않은 저사양 반도체칩을 중국에 공급했지만, 저사양 반도체칩도 AI와 슈퍼컴퓨팅에 사용될 수 있다는 지적이 제기되면서 \'국가안보 위험\'을 이유로 저사양 AI칩 등에 대해 추가 수출제한 조치를 가한 것이다. 이를 위해 상무부는 새로운 규칙에 기존 속도에 더해 AI칩에 대한 \'성능밀도\' 기준을 추가했다. 우선 300테라플롭스(teraflops·초당 1조 회의 연산 처리) 이상의 속도로 작동할 수 있는 데이터센터 칩을 중국에 판매하는 것을 금지한다. 또한 150~300테라플롭스의 속도를 가진 반도체 칩은 성능 밀도가 평방밀리미터(mm²)당 370기가플롭

In [11]:
# 경고를 출력하지 않도록 설정
import warnings
warnings.filterwarnings('ignore')

In [12]:
# 반복 실행할 횟수 생성
n = len(newsList)
print(n)

764


In [13]:
# newsList에 뉴스 본문을 저장할 열 추가
newsList['body'] = np.nan

# 반복문으로 네이버뉴스 본문 수집
for i in tqdm(range(n)):
    
    # 네이버뉴스 링크로 뉴스 본문 수집
    newsList['body'].iloc[i] = NaverNewsBody(nlink = newsList['nlink'].iloc[i])
    
    # 1초간 멈춤
    time.sleep(1)

  0%|          | 0/764 [00:00<?, ?it/s]

In [14]:
# newsList의 처음 5행 확인
newsList.tail()

,press,title,nlink,date,body
759,연합뉴스,"HBM 날개 단 SK하이닉스, 3분기 적자 줄였다…D램은 흑자전환(종합)",https://n.news.naver.com/mnews/article/001/001...,2023.10.26.,"3분기 영업손실 1조8천억원…매출은 전 분기 대비 24% 증가HBM, DDR5, L..."
760,노컷뉴스,"SK하이닉스, 3Q 영업손실 1.8조…적자 '1조' 줄였다",https://n.news.naver.com/mnews/article/079/000...,2023.10.26.,D램 흑자 전환 영향…HBM 등 주력제품 투자 확대\n\n\n\n연합뉴스SK하이닉스...
761,파이낸셜뉴스,[상보] SK하이닉스 D램 흑자전환... 적자폭 1兆 감소,https://n.news.naver.com/mnews/article/014/000...,2023.10.26.,경기도 이천시 SK하이닉스 본사. 뉴시스 [파이낸셜뉴스] 반도체 불황 장기화 속에서...
762,서울신문,[속보] SK하이닉스 3분기 영업손실 1조 8000억원…D램은 흑자 전환,https://n.news.naver.com/mnews/article/081/000...,2023.10.26.,SK하이닉스가 올해 3분기 영업손실을 1조 8000억원 규모로 줄였다. 고대역폭 메...
763,한국경제,SK하이닉스 D램 2분기 만에 흑자전환…3분기 영업손실 1.8조원,https://n.news.naver.com/mnews/article/015/000...,2023.10.26.,SK하이닉스가 올 3분기 1조8000억원에 가까운 영업적자를 기록했다. 아직 적자의...


In [16]:
# 결과를 CSV 파일로 저장
newsList.to_csv('naver_news_hinix.csv', index=False, encoding='utf-8-sig')